## Analyzing BGP Route Policies

Route policies for BGP are complex and error prone.  In this notebook we show how Batfish can be used to validate that your route policies behave as intended.  Batfish's `testRoutePolicies` question provides easy exploration of route-policy behavior through testing, while its `searchRoutePolicies` question actively searches for errors in a route policy by analyzing *all possible* behaviors.

To illustrate these capabilities, we'll use an example network with two border routers, named `border1` and `border2`.  Each router has a BGP session with a customer network and a BGP session with a provider network.  Our goal in this notebook is to validate the in-bound route policy from the customer, called `from_customer`, and the out-bound route policy to the provider, called `to_provider`.

The intent of the `from_customer` route policy is:
* filter private addresses
* only permit routes to known prefixes if they have the correct origin AS
* tag permitted routes with an appropriate community, and update the local preference

The intent of the `to_provider` route policy is:
* advertise all prefixes that we own
* advertise all customer routes
* don't advertise anything else
![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/validating-route-policies?pixel&useReferer)

We'll start, as usual, by initializing the example network that we will use in this notebook.

In [1]:
# Import packages and load questions
%run startup.py
load_questions()

# Initialize a network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/route-analysis"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

### Example 1: Filter private addresses in-bound


First, we'd like to ensure that the two `from_customer` route policies properly filter announcements for private addresses.  Traditionally you might validate this policy through some form of in-network testing.  With Batfish's `testRoutePolicies` question we can test a route policy's behavior without access to the live network.

In [2]:
inRoute1 = BgpRoute(network="10.0.0.0/24", originatorIp="4.4.4.4", originType="egp", protocol="bgp")
bfq.testRoutePolicies(policies="from_customer", direction="in", inputRoutes=[inRoute1]).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,from_customer,"BgpRoute(network='10.0.0.0/24', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",DENY,None,None
1,border2,from_customer,"BgpRoute(network='10.0.0.0/24', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",DENY,None,None


The first line of code above creates a `BgpRoute` object that specifies the input route announcement to use for testing, which in this case announces the prefix 10.0.0.0/24, has an originator IP that we arbitrarily chose, and has default values for other parts of the announcement.  The second line uses `testRoutePolicies` to test the behavior of the two `from_outside` route policies on this announcement.

The output of the question shows the results of the test.  In both border routers, the `from_outside` route policy properly denies this private address.

That result gives us some confidence in our route policies, but it is just a single test.  We can run `testRoutePolicies` on more private addresses to ensure they are denied.  However, we'd really like to know that *all* private addresses are denied by the two in-bound route maps.  To do that, we will use the `searchRoutePolicies` question and change our perspective a bit.  Instead of providing particular test announcements and seeing how they behave, we will ask Batfish to search for a route-policy behavior that violates our intent.  If we get one or more results, then we've found a bug.  If we get no results, then we can be sure that our configurations satisfy the intent, since Batfish explores *all possible* route-policy behaviors.

In [3]:
privateIps = ["10.0.0.0/8:8-32", "172.16.0.0/28:28-32", "192.168.0.0/16:16-32"]
inRoutes1 = BgpRouteConstraints(prefix=privateIps)
bfq.searchRoutePolicies(policies="from_customer", inputConstraints=inRoutes1, action="permit").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border2,from_customer,"BgpRoute(network='192.168.0.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='192.168.0.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['20:30'], localPreference=300, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[20:30]'), BgpRouteDiff(fieldName='localPreference', oldValue='0', newValue='300')])"


The first line above specifies the space of all private IP prefixes.  The second line creates a `BgpRouteConstraints` object, which is like the `BgpRoute` object we saw earlier but represents a set of announcements rather than a single one.  In this case, we are interested in all announcements that announce a prefix in `privateIps`.  Finally, the third line of code uses `searchRoutePolicies` to search for an announcement in the set `inRoutes` that is permitted by the `from_customer` route policy.

There are no results for `border1`, which means that its `from_customer` route policy properly filters *all* private addresses. However, the result for `border2` shows that its version of `from_customer` permits an announcement for the prefix 192.168.0.0/32.  The table also shows the route announcement that will be produced by `from_customer` in this case, along with a "diff" of the input and output announcements. Inspecting the configurations, we see that both route policies deny all announcements for prefixes in the prefix list `private-ips`, but the definitions of this prefix list differ in one line:

```
border1: ip prefix-list private-ips seq 15 permit 192.168.0.0/16 ge 16
border2: ip prefix-list private-ips seq 15 permit 192.168.0.0/16
```

The definition in `border2`'s config has accidentally omitted the `ge 16` clause and so only applies to /16 addresses.

### Example 2: Filter based on origin in-bound


Second, announcements for certain known prefixes should have the correct origin AS.  For purposes of this example, we assume that announcements for routes with a prefix in the range 5.5.5.0/24:24-32 should originate from the AS 44.  To check this, we will use `searchRoutePolicies` to ask the following question:  *Is there a permitted announcement for a prefix in the range 5.5.5.0/24:24-32 that does not originate from AS 44?*

In [4]:
knownPrefixes = "5.5.5.0/24:24-32"
badOrigin = "!/( |^)44$/"
inRoutes2 = BgpRouteConstraints(prefix=knownPrefixes, asPath=badOrigin)
bfq.searchRoutePolicies(policies="from_customer", inputConstraints=inRoutes2, action="permit").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


The first line above defines the known prefixes.  The second line specifies that we are interested in AS-paths that do not end in 44, using the same syntax that Batfish uses for regular-expression [specifiers](https://github.com/batfish/batfish/blob/master/questions/Parameters.md#set-of-enums-or-names).  Specifically, a regular expression is surrounded by `/` characters, and the leading `!` indicates that we are interested in AS-paths that do not match this regular expression.  Since there are no results, we can be sure that the our intent is satisfied.

### Example 3: Set attributes in-bound

Finally, let's make sure that our `from_outside` route policies tag each permitted route with the community 20:30 and set the local preference to 300.  To start, we can use `testRoutePolicies` to test this property on a specific route announcement.

In [5]:
inRoute3 = BgpRoute(network="2.0.0.0/8", originatorIp="4.4.4.4", originType="egp", protocol="bgp")
bfq.testRoutePolicies(policies="from_customer", direction="in", inputRoutes=[inRoute3]).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,from_customer,"BgpRoute(network='2.0.0.0/8', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='2.0.0.0/8', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=['20:30'], localPreference=300, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[20:30]'), BgpRouteDiff(fieldName='localPreference', oldValue='0', newValue='300')])"
1,border2,from_customer,"BgpRoute(network='2.0.0.0/8', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='2.0.0.0/8', originatorIp='4.4.4.4', originType='egp', protocol='bgp', asPath=[], communities=['20:30'], localPreference=300, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[20:30]'), BgpRouteDiff(fieldName='localPreference', oldValue='0', newValue='300')])"


The test behaves as intended:  the permitted route is tagged with community 20:30 and given local preference 300 on both routers.  This can be clearly seen in the `Output_Route` column, which shows the full output route announcement, as well as the `Difference` column, which shows the differences between the input and output route announcements.

However, we'd like to know that the route policies do the right thing for *all* permitted routes.  We can use `searchRoutePolicies` to check this.  First we'll check that all permitted routes are tagged with the community 20:30.  To check this property, we will leverage the ability of `searchRoutePolicies` not only to search for particular *input* announcements, but also to search for particular *output* announcements.  In this case, we will ask:  *Is there a permitted route whose output announcement is not tagged with community 20:30?*

In [6]:
outRoutes3a = BgpRouteConstraints(communities="!20:30")
bfq.searchRoutePolicies(policies="from_customer", action="permit", outputConstraints=outRoutes3a).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


There are no results, so we know that our intent is satisfied.  Now let's do a similar thing to check that the local preference is properly set.

In [7]:
outRoutes3b = BgpRouteConstraints(localPreference="!300")
bfq.searchRoutePolicies(policies="from_customer", action="permit", outputConstraints=outRoutes3b).answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border1,from_customer,"BgpRoute(network='5.5.5.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=[], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='5.5.5.0/32', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[{'asns': [44], 'confederation': False}], communities=['20:30'], localPreference=0, metric=0, sourceProtocol=None)","BgpRouteDiffs(diffs=[BgpRouteDiff(fieldName='communities', oldValue='[]', newValue='[20:30]')])"


There are no results for `border2`'s `from_customer` policy, so it is properly setting the local preference.  However, Batfish reports a violation for `border1`:  it does not set the local preference on announcements for our known prefix from Example 2 above.  A look at the configuration reveals that the `set local-preference` line is accidentally omitted from that clause of the policy.

### Example 4: Announce your own addresses out-bound


Ok, now let's validate the `to_provider` route policies.  First, they should allow all addresses that our network owns to be advertised.  These are addresses in the ranges 1.2.3.0/24:24-32 and 1.2.4.0/24:24-32.  We can use `searchRoutePolicies` to validate this property.  Specifically, we ask:  *Is there an announcement for an address that we own that is denied by `to_provider`?*

In [8]:
ownedSpace = ["1.2.3.0/24:24-32", "1.2.4.0/24:24-32"]
inRoutes4 = BgpRouteConstraints(prefix=ownedSpace)
bfq.searchRoutePolicies(policies="to_provider", inputConstraints=inRoutes4, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference


Since there are no results, this implies that no such announcement exists.  Hurray!

### Example 5: Announce your customers' routes out-bound

Next we will check that the `to_provider` route policies are properly announcing our customers' routes.  These are identified by announcements that are tagged with the community 20:30, as we saw earlier.

In [9]:
customerCommunities = "20:30"
inRoutes5 = BgpRouteConstraints(communities=customerCommunities)
bfq.searchRoutePolicies(policies="to_provider", inputConstraints=inRoutes5, action="deny").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border2,to_provider,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['20:30'], localPreference=0, metric=0, sourceProtocol=None)",DENY,None,None


There are no results for `border1`, so it permits announcement of all customer routes.  But `border2` has a bug.  A look at the configuration reveals a fat-finger error in the definition of the community list:

```
ip community-list cust_community permit 2:30
```

### Example 6: Don't advertise anything else out-bound


Last, we want to make sure that our `to_provider` route policies don't announce any routes other than the ones we own and the ones that our customers own.  We will use `searchRoutePolicies` to ask: *Is there a permitted route whose prefix is not one we own and which is not tagged with the community 20:30?*

In [10]:
inRoutes6 = BgpRouteConstraints(prefix=ownedSpace, complementPrefix=True, communities="!20:30")
bfq.searchRoutePolicies(policies="to_provider", inputConstraints=inRoutes6, action="permit").answer().frame()

,Node,Policy_Name,Input_Route,Action,Output_Route,Difference
0,border2,to_provider,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['2:30'], localPreference=0, metric=0, sourceProtocol=None)",PERMIT,"BgpRoute(network='0.0.0.0/0', originatorIp='0.0.0.0', originType='igp', protocol='bgp', asPath=[], communities=['2:30'], localPreference=0, metric=0, sourceProtocol=None)",BgpRouteDiffs(diffs=[])


The `complementPrefix` parameter above is used to indicate that we are interested in routes whose prefix is *not* in the set defined by `ownedSpace`.

Since there are no results for `border1` we can be sure that it is not advertising any routes that it shouldn't be.  We already saw in the previous example that `border2` accidentally advertises routes tagged with 2:30, and that error shows up again here.

### Current Status

The `testRoutePolicies` question supports all of the route policies that are parsed by Batfish.  The `searchRoutePolicies` question currently supports IOS, IOS XR, and Juniper route policies that use common forms of the following capabilities: matching on prefixes, communities, and AS paths; setting and deleting communities; and setting the metric and local preference. 

### Summary


In this notebook we showed you two ways to validate that your route policies meet your intent.  The `testRoutePolicies` question allows you to easily test the behavior of a route policy offline, without access to the live network.  The `searchRoutePolicies` question allows you to search for violations of intent, identifying concrete errors if they exist and providing strong correctness guarantees if not.